## Coming soon in `numba` 0.34

You can install the release candidate as of 07/09/2017 from the `numba` conda channel

```
conda install -c numba numba
```

In [ ]:
import numpy
from numba import njit

Define some reasonably expensive operation in a function.

In [ ]:
def do_trig(x, y):
    z = numpy.sin(x**2) + numpy.cos(y)
    return z

We can start with 1000 x 1000 arrays

In [ ]:
x = numpy.random.random((1000, 1000))
y = numpy.random.random((1000, 1000))

In [ ]:
%timeit do_trig(x, y)

Now let's `jit` this function.  What do we expect to get out of this?  Probably nothing, honestly.  As we've seen, `numpy` is pretty good at what it does.

In [ ]:
do_trig_jit = njit()(do_trig)

In [ ]:
%timeit do_trig_jit(x, y)

Maybe a _hair_ slower than the bare `numpy` version. So yeah, no improvement. 

### BUT

Starting in version 0.34, with help from the Intel Parallel Accelerator team, you can now pass a `parallel` keyword argument to `jit` and `njit`. 

Like this:

In [ ]:
do_trig_jit_par = njit(parallel=True)(do_trig)

How do we think this will run?

In [ ]:
%timeit do_trig_jit_par(x, y)

Not bad -- around a 3x speedup for a single line?

And what if we unroll the array operations like we've seen before?  Does that help us out?

In [ ]:
@njit
def do_trig(x, y):
    z = numpy.empty_like(x)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i, j] = numpy.sin(x[i, j]**2) + numpy.cos(y[i, j])
    return z

In [ ]:
%timeit do_trig(x, y)

Hmm, that's actually a hair faster than before.  Cool!

Now let's parallelize it!

In [ ]:
@njit(parallel=True)
def do_trig(x, y):
    z = numpy.empty_like(x)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i, j] = numpy.sin(x[i, j]**2) + numpy.cos(y[i, j])
    return z

In [ ]:
%timeit do_trig(x, y)

What happened?

Well, automatic parallelization is a _pretty hard_ problem.  (This is a massive understatement).

Basically, parallel `jit` is "limited" to working on array operations, so in this case, unrolling loops will hurt you. 
Blarg.

### FAQ that I just made up

- Why didn't you tell us about this before?

It is brand new. The numba team is great, but have a really bad habit of releasing new features 5-10 days before I run a tutorial.

- Is regular `jit` just dead now?

It honestly might be. I've only started playing around with it but I haven't seen any speed _hits_ for using it when there are no array operations to operate on.

- Is all of that stuff about `vectorize` just useless now?

Short answer: no.  Long answer: Let's check it out!

In [ ]:
from numba import vectorize
import math

Recall that we define the function as if it operates on scalars, then apply the vectorize decorator.

In [ ]:
@vectorize
def do_trig_vec(x, y):
    z = math.sin(x**2) + math.cos(y)
    return z

In [ ]:
%timeit do_trig_vec(x, y)

A little faster, but roughly equivalent to the base `numpy` and `jit` versions. Now let's type our inputs and run it in `parallel`

In [ ]:
@vectorize('float64(float64, float64)', target='parallel')
def do_trig_vec_par(x, y):
    z = math.sin(x**2) + math.cos(y)
    return z

In [ ]:
%timeit do_trig_vec_par(x, y)

Yowza! So yeah, `vectorize` is still the best performer when you have element-wise operations, but if you have a big mess of stuff that you just want to speed up, then parallel `jit` is an awesome and easy way to try to boost performance.

In [ ]:
a = x
b = y
c = numpy.random.random((a.shape))

In [ ]:
%%timeit
b**2 - 4 * a * c

In [ ]:
def discrim(a, b, c):
    return b**2 - 4 * a * c

In [ ]:
discrim_vec = vectorize()(discrim)

In [ ]:
%timeit discrim_vec(a, b, c)

In [ ]:
discrim_vec_par = vectorize('float64(float64, float64, float64)', target='parallel')(discrim)

In [ ]:
%timeit discrim_vec_par(a, b, c)

In [ ]:
discrim_jit = njit()(discrim)

In [ ]:
%timeit discrim_jit(a, b, c)

In [ ]:
discrim_jit_par = njit(parallel=True)(discrim)

In [ ]:
%timeit discrim_jit_par(a, b, c)